In [35]:
import xgboost as xgb
import pandas as pd
import numpy as np

#train = pd.read_csv('../dataset/pre/train.csv')
labels = train["label"]

features = train.drop(['label','conversionTime'], axis=1)
data_dmat = xgb.DMatrix(data=features, label=labels)

params={"max_depth":5, "min_child_weight":2, "eta": 0.1, "subsamples":0.9, "colsample_bytree":0.8, "objective" : "binary:logistic", "eval_metric": "logloss", "seed": 2333}
rounds = 50

result = xgb.cv(params=params, dtrain=data_dmat, num_boost_round=rounds, early_stopping_rounds=5, as_pandas=True, seed=2333)
print result

    test-logloss-mean  test-logloss-std  train-logloss-mean  train-logloss-std
0            0.607393          0.000025            0.607392           0.000036
1            0.537263          0.000020            0.537261           0.000101
2            0.478927          0.000067            0.478925           0.000107
3            0.429806          0.000079            0.429802           0.000137
4            0.388073          0.000102            0.388069           0.000159
5            0.352333          0.000110            0.352329           0.000193
6            0.321558          0.000133            0.321553           0.000207
7            0.294916          0.000171            0.294909           0.000206
8            0.271789          0.000210            0.271782           0.000204
9            0.251660          0.000212            0.251651           0.000236
10           0.234100          0.000250            0.234090           0.000228
11           0.218736          0.000258            0

In [1]:
import pandas as pd

In [2]:
test = pd.read_csv('../dataset/pre/test.csv')
train = pd.read_csv('../dataset/pre/train.csv')

In [16]:
test['clickDate'] = test['clickTime'].apply(lambda x : x/10000)

In [3]:
train['clickDate'] = train['clickTime'].apply(lambda x : x/10000)

In [4]:
train  = train[(train.clickDate == 21) 
              | (train.clickDate == 22)
              | (train.clickDate == 23)
              | (train.clickDate == 24)
              | (train.clickDate == 25)
              | (train.clickDate == 26)
              | (train.clickDate == 27)
              | (train.clickDate == 28)
              | (train.clickDate == 29)]

In [5]:
ad = pd.read_csv('../dataset/pre/ad.csv')
app_categories = pd.read_csv('../dataset/pre/app_categories.csv')
position = pd.read_csv('../dataset/pre/position.csv')
user = pd.read_csv('../dataset/pre/user.csv')
user_app_actions = pd.read_csv('../dataset/pre/user_app_actions.csv')
user_installedapps = pd.read_csv('../dataset/pre/user_installedapps.csv')

In [6]:
train['label'].value_counts()

0    2582920
1      67047
Name: label, dtype: int64

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338489 entries, 0 to 338488
Data columns (total 8 columns):
instanceID          338489 non-null int64
label               338489 non-null int64
clickTime           338489 non-null int64
creativeID          338489 non-null int64
userID              338489 non-null int64
positionID          338489 non-null int64
connectionType      338489 non-null int64
telecomsOperator    338489 non-null int64
dtypes: int64(8)
memory usage: 20.7 MB


In [9]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

In [10]:
feature = [x for x in train.columns if x not in ['label','conversionTime']]
target = 'label'

In [11]:
model = RandomForestClassifier()

In [13]:
model.fit(train[feature],train[target])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [14]:
model.feature_importances_

array([ 0.38982326,  0.08177365,  0.41528333,  0.07937586,  0.0038321 ,
        0.01290055,  0.01701125])

In [17]:
predict1 = model.predict(test[feature])
out1 = model.predict_proba(test[feature])

In [18]:
pd.DataFrame(predict1)[0].value_counts()

0    337508
1       981
Name: 0, dtype: int64

In [19]:
model2 = xgb.XGBClassifier()

In [20]:
model2.fit(train[feature],train[target])

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [21]:
model2.feature_importances_

array([ 0.00714286,  0.37285715,  0.00142857,  0.48714286,  0.11285714,
        0.01857143,  0.        ], dtype=float32)

In [22]:
predict2 = model2.predict(test[feature])
out2 = model2.predict_proba(test[feature])

In [23]:
pd.DataFrame(predict2)[0].value_counts()

0    338489
Name: 0, dtype: int64

In [24]:
out1[:,1]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [25]:
out2[:,1]

array([ 0.0143971 ,  0.01710395,  0.0057922 , ...,  0.02308859,
        0.00854214,  0.00511939], dtype=float32)

In [28]:
predict = out1[:,1]*0.6 + out2[:,1]*0.4

In [39]:
predict = out2[:,1]

In [40]:
submission = pd.DataFrame(test['instanceID'])

In [41]:
submission['pro'] = predict

In [42]:
submission.to_csv('../output/submission.csv',index=False)